In [1]:
import sys
sys.path.append('..') # to make modules importable

In [2]:
import pickle
from tqdm.notebook import tqdm
import pandas as pd
from dicts import var_vocab, args, topic_to_russian
from classes.Preprocessor import Preprocessor
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("../data/df.csv", low_memory=False)
df.sample(3)

,document.id,source,stage,source_text,lemm_text,city,region,date,data,assessor,...,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id,text_sentiment
36980,663079512,iqbuzz,3,Елена. Сдается ком. 15 м кв + застекленная лод...,бесплатный объявление москва елена сдаваться к...,Москва,Москва,2014-12-01,2017-03-31 01:25:33,alyonam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,663079512,0.0
2826,616278028,iqbuzz,2,Татарстан для татар ! Башкортостан для Башкорт...,татарстан татарин башкортостан башкорт чувашия...,Уфа,Пермский край,2014-09-26,2016-10-20 14:49:19,skuchilina,...,no,3.0,irrel,3.0,irrel,1.0,no,NaN,аланы_142,0.0
21449,926873229,iqbuzz,2,"ИМХО, так их правительству походу безразлично ...",николай имхо правительство поход безразличный ...,Санкт-Петербург,Санкт-Петербург,2015-06-05,2016-09-24 23:56:04,an_men,...,yes,2.0,high,3.0,irrel,1.0,no,NaN,прибалт_111,-1.0


In [4]:
ids = df['document.id'].unique()
RANDOM_STATE = 42
process_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=RANDOM_STATE)
train_ids, validate_ids = train_test_split(process_ids, train_size=0.75, random_state=RANDOM_STATE)

train = df.loc[df['document.id'].isin(train_ids)]
test = df.loc[df['document.id'].isin(test_ids)]
validate = df.loc[df['document.id'].isin(validate_ids)]
train.shape, test.shape, validate.shape  # percents are ≈ (60%, 20%, 20%)

((47999, 62), (16170, 62), (16281, 62))

In [5]:
with open("../data/gpt_scores.pkl", "rb") as f:
    res = pickle.load(f)

In [6]:
best_model = None
best_score = 0
for model, score in res.items():
    if score > 1170:
        print(f"{model} score is {score}")
    if best_score < score:
        best_score = score
        best_model = model

epoch_8_num_54_loss_0.7994186667966574 score is 1174
epoch_5_num_52_loss_0.7315569369028031 score is 1179
epoch_7_num_10_loss_0.7702571042826478 score is 1173
epoch_2_num_54_loss_0.6606334624549909 score is 1173
epoch_3_num_10_loss_0.6640205195252324 score is 1171


In [7]:
print(f"{best_model} score is {best_score}")

epoch_5_num_52_loss_0.7315569369028031 score is 1179


In [8]:
preprocessor = Preprocessor(df=df, args=args, var_vocab=var_vocab, topic_to_russian=topic_to_russian)

In [9]:
from torch.utils.data import DataLoader
from gpt_classes.GPTDataset import GPTDataset
from gpt_classes.GPTModel import GPTModel
import torch

In [10]:
dataset = GPTDataset(validate, args=args, preprocessor=preprocessor)

In [11]:
loader = DataLoader(dataset, batch_size=16)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GPTModel(device).to(device)

In [12]:
# best_model = 'epoch_3_num_10_loss_0.6640205195252324'
model.load_state_dict(torch.load(f"../models/{best_model}", map_location=torch.device(device)))
print(device, best_model)

cpu epoch_5_num_52_loss_0.7315569369028031


In [13]:
all_generated = []
all_texts = []
all_desc = []

In [14]:
with torch.no_grad():
    for i, (data, cut_data) in enumerate(tqdm(loader), 1):
        outputs = model(data, cut_data)

        for el in data:
            text, descr = el.split("Описание: ", 1)
            all_texts.append(text)
            all_desc.append(descr)

        generated = model.my_generate(cut_data)
        all_generated.extend(generated)
    

  0%|          | 0/178 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (569 > 512). Running this sequence through the model will result in indexing errors


In [21]:
i = 0
for gen, text, descr in zip(all_generated, all_texts, all_desc):
    print(f"\n{gen}\n======\n")
    i += 1
    if i >= 5:
        break


Задание: Сгенерируй описание следующего текста, оцени тональность текста, оцени наличие этнического конфликта, наличие позитивного взаимодействия этничностей, мнение о абхаз в тексте, является ли этничность абхаз доминирующей, является ли этничность абхаз агрессором, является ли этничность абхаз опасной.
Текст: Для примера приведу по датам ключевые моменты Грузино-Абхазкой войны в хронологическом порядке. Количество договоренностей о прекращении огня можете посчитать сами. К чему эти договоренности привели ясно из хроники. Роль России тоже видна не вооруженным глазом.
Описание: этот текст является негативным, в этническом составе Грузии есть этнические конфликты, этногенез, но они не носят локальный характер, а имеют место быть, это не означает, что в грузии не будет создано правительство или народная партия, которое будет диктовать свою волю грузинам, абхазы не хотят иметь правительство, у них нет договора с Россией, они хотят независимости своей республики, и это их естественное сос

In [16]:
from classes.MyModel import MyModel
import numpy as np
from sklearn.metrics import f1_score

In [17]:
bert = MyModel('cpu')
bert.load_state_dict(torch.load('../data/best_model'))

<All keys matched successfully>

In [43]:
def calculate_score(y_true, y_pred):
    y_true = y_true.cpu().detach().numpy()
    y_pred = np.argmax(y_pred.cpu().detach().numpy(), axis=1)

    f1 = f1_score(y_true, y_pred)
    print(f'f1 score: {f1:.5f}')

    return f1

In [32]:
def validate():
    all_outputs = torch.empty(0).to('cpu')

    with torch.no_grad():
        for i, input in enumerate(all_generated, 1):
            input = "Текст:" + input.split("Текст:")[1] # "Задание: ... Текст: ... Описание: ... -> Текст: ... Описание: ..."
            outputs = bert(input)
            all_outputs = torch.cat((all_outputs, outputs.to('cpu')), dim=0)
    return all_outputs

In [34]:
outputs = validate()

In [53]:
y_true = torch.zeros(len(all_generated)).to('cpu')

In [59]:
y_pred = np.argmax(outputs.cpu().detach().numpy(), axis=1)
print(f"{np.sum(1 - y_pred)} predicted to be true descriptions out of {len(y_true)}")

1451 predicted to be true descriptions out of 2839


In [60]:
f1_score(1 - y_true, 1 - y_pred)

0.6764568764568765